# Advent of Code 2023

## Day 1

### Part 1

In [3]:
with open('inputs/day1.txt') as f:
    s = f.read()[:-1]

def parse(n):
    return int(n[0] + n[-1])

sum([parse([c for c in n if c.isnumeric()]) for n in s.split('\n')])

54634

### Part 2

In [4]:
numeral_map = {'one':'1',
               'two':'2',
               'three':'3',
               'four':'4',
               'five':'5',
               'six':'6',
               'seven':'7',
               'eight':'8',
               'nine':'9'}
numerals = list(numeral_map.keys()) + list(numeral_map.values())

def parse_numeral(n):
    return numeral_map[n] if n in numeral_map else n


def extract_number(s):
    indices_left = {n:s.find(n) for n in numerals if s.find(n) >= 0}
    indices_right = {n:s.rfind(n) for n in numerals if s.find(n) >= 0}
    leftmost = parse_numeral(min(indices_left,key=indices_left.get))
    rightmost = parse_numeral(max(indices_right,key=indices_right.get))
    return int(leftmost + rightmost)


sum([extract_number(n) for n in s.split('\n')])

53855

## Day 2

### Part 1

In [1]:
with open('inputs/day2.txt') as f:
    s = f.read()[:-1]

def parse_round(round):
    return {cubes.split(' ')[1]: int(cubes.split(' ')[0]) for cubes in round.split(', ')}


def parse_game(game):
    return [parse_round(round) for round in game.split(': ')[1].split('; ')]


def max_color_value(game, color):
    return (color, max([round.get(color,0) for round in game]))


def max_values(game):
    return dict(map(lambda c: max_color_value(game,c), ['red', 'green', 'blue']))


def is_valid(game,reds,greens,blues):
    maxes = max_values(game)
    return maxes.get('red',0) <= reds and maxes.get('green',0) <= greens and maxes.get('blue',0) <= blues


games = [parse_game(game) for game in s.split('\n')]
sum([i+1 for i,game in enumerate(games) if is_valid(game,12,13,14)])

2593

### Part 2

In [3]:
from functools import reduce

def game_power(game):
    return reduce(lambda a,b: a*b, max_values(game).values())


sum(map(game_power, games))

54699

## Day 3

### Part 1

In [114]:
from functools import reduce

with open('inputs/day3.txt') as f:
    s = f.read()[:-1].split('\n')

symbols = [(x,y) for y in range(len(s)) for x in range(len(s[0])) if not (s[y][x] == '.' or s[y][x].isnumeric())]
numbers = [(x,y) for y in range(len(s)) for x in range(len(s[0])) if s[y][x].isnumeric()]
neighbours = [(-1,-1), (-1,0), (-1,1), (0,-1), (0,1), (1,-1), (1,0), (1,1)]

def explore(reachable, current, target):
    (x,y) = current
    neighborhood = [(x+xd,y+yd) for (xd,yd) in neighbours]
    for coord in neighborhood:
        if coord in target and coord not in reachable:
            reachable.add(coord)
            explore(reachable, coord, target)
    return reachable


reachable = reduce(lambda acc,sym: explore(acc, sym, numbers), symbols, set())
number_starts = [(x,y) for (x,y) in reachable if (x-1,y) not in reachable]

def read_number(current, acc, reachable):
    (x,y) = current
    if current in reachable:
        return read_number((x+1,y), acc*10+int(s[y][x]), reachable)
    return acc


sum(map(lambda x: read_number(x,0,reachable), number_starts))

521515

### Part 2

In [116]:
symbols = [(x,y) for y in range(len(s)) for x in range(len(s[0])) if s[y][x] == '*']
reachable = {sym: explore(set(), sym, numbers) for sym in symbols}
numbers = list(reduce(lambda acc, v: acc.union(v), reachable.values(), set()))
number_starts = [(x,y) for (x,y) in numbers if (x-1,y) not in numbers]
reachable_number_starts = {sym: [n for n in reachable[sym] if n in number_starts] for sym in reachable}
gears = [reduce(lambda a,b: a*b, map(lambda x: read_number(x,0,numbers), num_starts), 1) 
         for num_starts in reachable_number_starts.values() if len(num_starts) == 2]

sum(gears)

69527306

## Day 4

### Part 1

In [39]:
with open('inputs/day4.txt') as f:
    s = f.read()[:-1]

input = [line.replace('  ', ' ').split(': ')[1].split(' | ') for line in s.split('\n')]
cards = [(list(map(int,card[0].split(' '))), list(map(int,card[1].split(' ')))) for card in input]
winning = [(len([x for x in card[1] if x in card[0]])) for card in cards]
sum([2 ** (card-1) for card in winning if card > 0])

21485

### Part 2

In [40]:
card_count = {i: 1 for i in range(len(cards))}
for i in range(len(cards)):
    for j in range(1, winning[i]+1):
        card_count[i+j] += card_count[i]

sum(card_count.values())

11024379

## Day 5

### Part 1

In [147]:
with open('inputs/day5.txt') as f:
    s = f.read()[:-1]

seeds = [int(seed) for seed in s.split('\n\n')[0].split(' ')[1:]]
maps = [[tuple(int(n) for n in m.split(' ')) for m in sec.split('\n')[1:]] for sec in s.split('\n\n')[1:]]

def attempt_map(n, map):
    destination_start, source_start, range_length = map
    if n >= source_start and n < source_start + range_length:
        return (destination_start + n - source_start, True)
    else:
        return (n, False)

def map_level(n,mapping):
    match = False
    i = -1
    while not match and i < len(mapping) - 1:
        i += 1
        result, match = attempt_map(n, mapping[i])
    return (result, i)


def map_full(n, levels):
    result = (n,None)
    for level in levels:
        result = map_level(result[0],level)
    return result

min([map_full(seed, maps)[0] for seed in seeds])

484023871

### Part 2

In [168]:
from math import ceil

seed_ranges = [(seeds[i], seeds[i+1]) for i in range(0, len(seeds), 2)]
lowest_start = min([map_full(seed_range[0], maps) for seed_range in seed_ranges])

def find_buckets(seed_range, levels, buckets):
    (start, length) = seed_range
    if length <= 1:
        return buckets
    (result_1, last_map_1) = map_full(start, levels)
    (result_2, last_map_2) = map_full(start+length-1, levels)
    if last_map_1 not in buckets or buckets[last_map_1] > result_1:
        buckets[last_map_1] = result_1
    if last_map_2 not in buckets or buckets[last_map_2] > result_2:
        buckets[last_map_2] = result_2
    if last_map_1 != last_map_2:
        h1 = find_buckets((start, length//2), levels, buckets)
        h2 = find_buckets((start+length//2, ceil(length/2)), levels, buckets)
        for b in h1:
            if b not in buckets or buckets[b] > h1[b]:
                buckets[b] = h1[b]
        for b in h2:
            if b not in buckets or buckets[b] > h2[b]:
                buckets[b] = h2[b]
    return buckets


buckets = {}
for sr in seed_ranges:
    buckets = find_buckets(sr, maps, buckets)

min(buckets.values())

46294175